In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.io as scio
from scipy import optimize
from scipy.special import expit

%matplotlib inline

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def hypothesis(theta, X):
    return expit(np.dot(X, theta))

In [4]:
def cost(theta, X, y, penalty = 0.):
    m = y.size
    h = hypothesis(theta, X)
    prob1 = np.log(h).dot(y.T)
    prob2 = np.log(1.0 - h).dot(1 - y.T)
    cost = -1. * (prob1 + prob2)/m
    reg = (theta.T.dot(theta)) * (penalty/ (2*m))
    
    return cost + reg

In [5]:
def costGradient(theta, X, y, penalty = 0.):
    m = y.size
    h = hypothesis(theta, X)
    
    ##grad = (1ambda/m) * X.T * (h-y)
    
    error = h - y
    
    reg = theta[1:] * (penalty/m)
    #grad = (1./m) * X.T.dot(error)
    grad = (1./m)*np.dot(X.T,error)
    
    grad[1:] = grad[1:] + reg
    return grad

In [6]:
def optimizeRegLR(theta, X, y, penalty = 0.):
    result = optimize.fmin_cg(cost, fprime=costGradient, x0=theta,\
                             args=(X, y, penalty), maxiter=50, disp=False,\
                              full_output=True)
    return result[0], result[1]

In [7]:
def optimizeAllThetaLr(X, y):
    penalty = 0.
    initial_theta = np.zeros((X.shape[1],1)).reshape(-1)
    theta_cap = np.zeros((10, X.shape[1]))
    
    for i in range(10):
        clazz = i if i else 10
        y_processed = np.array([1 if x==clazz else 0 for x in y])
        theta, cost = optimizeRegLR(initial_theta, X, y_processed, penalty)
        theta_cap[i, :] = theta
        
    print ('Complete')
    return theta_cap

In [8]:
ex_theta = np.zeros((8,1)).reshape(-1)
ex_theta_cap = np.zeros((10, 8))

print(ex_theta)
print(ex_theta_cap)

[ 0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [9]:
def predictOneVsAll(theta, img_x):
    classes = [10] + list(range(1,10))
    outputs = [0] * len(classes)
    
    for i in range(len(classes)):
        outputs[i] = hypothesis(theta[i], img_x)
    return classes[np.argmax(np.array(outputs))]

In [10]:
data = scio.loadmat('data/ex3data1.mat')
print(data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011', '__version__': '1.0', '__globals__': [], 'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), 'y': array([[10],
       [10],
       [10],
       ..., 
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)}


In [11]:
X, y = data['X'], data['y']

##Shape of X
print('Shape of X: ', X.shape)
print('Shape of y: ', y.shape)

print('Shape of a single input row', X[0].shape)

Shape of X:  (5000, 400)
Shape of y:  (5000, 1)
Shape of a single input row (400,)


In [12]:
#Add the xo input to X
X = np.insert(X,0,1,axis=1)

In [13]:
Theta = optimizeAllThetaLr(X, y)

Complete


In [14]:
n_correct, n_total = 0., 0.
incorrect_indices = []
for irow in range(X.shape[0]):
    n_total += 1
    if predictOneVsAll(Theta, X[irow]) == y[irow]: 
        n_correct += 1
    else: incorrect_indices.append(irow)
print ("Training set accuracy: %0.1f%%"%(100*(n_correct/n_total)))

Training set accuracy: 95.3%


In [15]:
theta0 = Theta[0]
print(theta0.shape)
print(theta0)

(401,)
[ -8.01443255e+00   0.00000000e+00   0.00000000e+00  -2.36195603e-07
  -1.84560850e-05   2.29260267e-04   2.20826477e-03   7.04458467e-04
  -8.72787420e-04  -1.88107763e-03  -2.66016458e-03  -4.69965645e-04
   5.67973163e-04   1.00121090e-03   8.98327604e-03   8.02255426e-03
  -1.41482824e-05   7.88516776e-06   3.91333205e-07  -4.34865139e-07
   0.00000000e+00  -2.23406730e-07  -1.28783100e-05  -3.54989405e-05
   2.15514409e-03   2.09238564e-03  -2.09761785e-02   9.56062607e-04
   5.46897825e-03  -2.70403417e-03   1.87881249e-02   2.03509459e-02
   2.45271683e-02   1.63060745e-02  -6.62508858e-02  -8.79274686e-02
  -1.68704002e-02   9.36357766e-03   9.29005889e-03  -3.78965075e-04
   6.53191911e-06   1.97668049e-06   5.47812175e-05   1.73556972e-04
  -8.43780110e-03  -3.49191445e-02  -1.08321314e-01  -1.84232494e-01
  -2.86914665e-01  -3.94516691e-01  -4.27653881e-01  -6.16599689e-01
  -8.20528399e-01  -6.71981327e-01  -1.05575547e+00  -5.54290613e-01
   2.24356783e-01  -4.83806